In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model as tf_load_model
from tensorflow.keras.callbacks import EarlyStopping

import matplotlib.pyplot as plt

import datetime
import os

In [2]:
IMG_SIZE = 128
BATCH_SIZE = 32

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,         
    rotation_range=45,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    brightness_range=[0.7, 1.3],
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest',
    validation_split=0.1
)

train_generator = train_datagen.flow_from_directory(
    'raw-img',           
    target_size=(IMG_SIZE, IMG_SIZE),     
    batch_size=BATCH_SIZE,            
    class_mode='categorical',      
    subset='training'         
)

validation_generator = train_datagen.flow_from_directory(
    'raw-img',          
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'   
)

early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=5,              
    restore_best_weights=True,  
    verbose=1                
)

In [4]:
def build_model(filtersList, denseUnits, shape, activation):
    
    model = models.Sequential()
    
    for i, filters in enumerate(filtersList):
        if i == 0:
            model.add(layers.Conv2D(filters, (3, 3), activation=activation, input_shape=shape, padding='same'))
        else:
            model.add(layers.Conv2D(filters, (3, 3), activation=activation, padding='same'))
        model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Flatten())
    
    for dense_units in denseUnits:
        model.add(layers.Dense(dense_units, activation=activation))
    
    model.add(layers.Dense(3, activation='softmax'))

    model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    
    return model


In [ ]:
TrainingModels = []
Repetitions = 10
EpochList = [20, 40, 50]

TrainingModels.append(build_model([128, 64, 32, 16, 8, 4, 2], [128, 64, 32, 16, 8, 4], (IMG_SIZE, IMG_SIZE, 3), 'relu'))
TrainingModels.append(build_model([128, 64, 32, 16, 8, 4, 2], [64, 32, 16, 8, 4], (IMG_SIZE, IMG_SIZE, 3), 'relu'))
TrainingModels.append(build_model([128, 64, 32, 16, 8, 4, 2], [32, 16, 8, 4], (IMG_SIZE, IMG_SIZE, 3), 'relu'))

In [ ]:
log_path = './training_log.txt'

if not os.path.exists(log_path):
    with open(log_path, 'w') as log_file:
        log_file.write("Timestamp,Model,Epochs,Repetition,Validation Loss,Validation Accuracy\n")

for model_index, model in enumerate(TrainingModels):
    for epoch in EpochList:
        for repetition in range(1, Repetitions + 1):

            history = model.fit(
                train_generator,
                validation_data=validation_generator,
                epochs=epoch,
                verbose=1,  # Show progress
                callbacks=[early_stopping]
            )

            timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

            validation_metrics = model.evaluate(validation_generator, verbose=1)
            val_loss, val_accuracy = validation_metrics[0], validation_metrics[1]

            if val_accuracy > 0.8:
                model_path = f'./models/model_{model_index + 1}_epochs_{epoch}_rep_{repetition}_{timestamp}.keras'
                model.save(model_path)

            with open(log_path, 'a') as log_file:
                log_file.write(f"{timestamp},Model_{model_index + 1},{epoch},{repetition},{val_loss:.4f},{val_accuracy:.4f}\n")